In [16]:
import requests
import pandas as pd
import datetime

headers = {"User-Agent": "russ@sunriseanalysis.com"}

ticker = "AAPL"
concept = "NetIncomeLoss"

In [17]:
def get_cik(ticker):
    """
    Get the CIK for a given ticker.
    Args:
        ticker (str): The ticker symbol of the company.
    Returns:
        str: The CIK for the company.
    """
    ticker = ticker.upper()
    headers = {"User-Agent": "russ@sunriseanalysis.com"}
    # Get the tickers JSON file
    tickers_json = requests.get(
        "https://www.sec.gov/files/company_tickers.json", headers=headers
    )
    # Create a DataFrame of the tickers
    cik_df = pd.DataFrame.from_dict(tickers_json.json(), orient="index")
    cik_df["cik_str"] = cik_df["cik_str"].astype(str).str.zfill(10)
    # Get the CIK for the given ticker
    cik = cik_df[cik_df["ticker"] == ticker]["cik_str"].values[0]
    return cik

In [18]:
cik = get_cik(ticker)

In [19]:
concept_url = (
    f"https://data.sec.gov/api/xbrl/companyconcept/CIK{cik}/us-gaap/{concept}.json"
)
concept_df = pd.DataFrame.from_dict(requests.get(concept_url, headers=headers).json())

In [20]:
concept_units = pd.DataFrame.from_dict(concept_df["units"]["USD"])
for col in ["start", "end", "filed"]:
    concept_units[col] = pd.to_datetime(concept_units[col])
df = concept_units
df["time_delta"] = df["end"] - df["start"]
df["val"] = df["val"] / 1_000_000
df.rename(columns={"val": "value(millions)"}, inplace=True)

annual_data = df[df["time_delta"] > pd.Timedelta(days=350)]
df = df.drop(annual_data.index)
annual_data = annual_data.drop_duplicates(subset="value(millions)", keep="last")
annual_data = annual_data.dropna(subset="frame")
annual_data = annual_data.drop(
    columns=["time_delta", "filed", "form", "fy", "fp"]
).reset_index(drop=True)

annual_data.rename(columns={"frame": "fiscal_year"}, inplace=True)
extra_data = df[df["time_delta"] > pd.Timedelta(days=115)]
df = df.drop(extra_data.index)
extra_data.reset_index(drop=True, inplace=True)
df = df.drop_duplicates(subset="value(millions)", keep="last")
df = df.drop(columns=["time_delta", "filed", "form", "fy", "fp", "accn"])
df = df.dropna(subset=["frame"])
df = df.reset_index(drop=True)

In [21]:
def map_to_quarter(end_date):
    month_diff = (end_date.month - end_month + 9) % 12
    if 0 <= month_diff < 3:
        return 1
    elif 3 <= month_diff < 6:
        return 2
    elif 6 <= month_diff < 9:
        return 3
    else:  # 9, 10, 11
        return 4


end_month = annual_data.iloc[0]["end"].month
df["quarter"] = df["end"].apply(map_to_quarter)

In [22]:
start_year = int(df["frame"].min()[2:6])  # extract start year from the minimum frame
end_year = int(df["frame"].max()[2:6])  # extract end year from the maximum frame
frames = [
    f"CY{year}Q{quarter}"
    for year in range(start_year, end_year + 1)
    for quarter in range(1, 5)
]
df_full = pd.DataFrame({"frame": frames})
# Step 2: Merge the original DataFrame with the new DataFrame
quarterly_data = pd.merge(df_full, df, how="left", on="frame")

# Step 3: Sort the DataFrame by frame
quarterly_data = quarterly_data.sort_values(by="frame").reset_index(drop=True)

In [23]:
quarterly_data["next_start"] = quarterly_data["start"].shift(-1)
quarterly_data["prev_end"] = quarterly_data["end"].shift()

# For missing quarters, set the 'start' date as the day after the end of the previous quarter,
# and the 'end' date as the day before the start of the next quarter
quarterly_data.loc[quarterly_data["start"].isna(), "start"] = quarterly_data[
    "prev_end"
] + pd.Timedelta(days=1)
quarterly_data.loc[quarterly_data["end"].isna(), "end"] = quarterly_data[
    "next_start"
] - pd.Timedelta(days=1)

# Drop the 'next_start' and 'prev_end' columns as they are no longer needed
quarterly_data.drop(["next_start", "prev_end"], axis=1, inplace=True)

In [24]:
today = datetime.datetime.today()
# Calculate the start date of the current quarter
if today.month < 4:
    quarter_start = datetime.datetime(today.year - 1, 12, 1)
elif today.month < 7:
    quarter_start = datetime.datetime(today.year, 3, 1)
elif today.month < 10:
    quarter_start = datetime.datetime(today.year, 6, 1)
else:
    quarter_start = datetime.datetime(today.year, 9, 1)
# Remove rows where the 'start' date is on or after the start of the current quarter
quarterly_data = quarterly_data[quarterly_data["start"] < quarter_start]

In [25]:
# Define a helper function to fill the quarter column
def fill_quarter(row, df):
    if pd.isnull(row["quarter"]):
        previous_index = row.name - 1
        next_index = row.name + 1 if row.name < len(df) - 1 else row.name

        if previous_index >= 0 and not pd.isnull(df.loc[previous_index, "quarter"]):
            previous_quarter = df.loc[previous_index, "quarter"]
            row["quarter"] = 1 if previous_quarter == 4 else previous_quarter + 1
        elif not pd.isnull(df.loc[next_index, "quarter"]):
            next_quarter = df.loc[next_index, "quarter"]
            row["quarter"] = 4 if next_quarter == 1 else next_quarter - 1
    return row


# Apply the function to each row in the DataFrame
quarterly_data = quarterly_data.apply(
    lambda row: fill_quarter(row, quarterly_data), axis=1
)
quarterly_data["quarter"] = quarterly_data["quarter"].astype(int)

In [26]:
def find_fiscal_year(date, annual_data):
    for idx, row in annual_data.iterrows():
        if row["start"] <= date <= row["end"]:
            return row["fiscal_year"]

    # If the date is after the end of the last fiscal year in annual_data, return the next fiscal year
    last_fiscal_year_end = annual_data["end"].max()
    if date > last_fiscal_year_end:
        # Extract the last year, increment it by one, and return it as a fiscal year
        last_year = int(annual_data.iloc[-1]["fiscal_year"][2:])
        next_year = last_year + 1
        return "CY" + str(next_year)

    return None


# Apply the function to the 'end' column in 'df'
quarterly_data["fiscal_year"] = quarterly_data["end"].apply(
    find_fiscal_year, args=(annual_data,)
)

In [27]:
def fill_missing_quarters(df, annual_data):
    # Iterate over each row in the dataframe
    for idx, row in df.iterrows():
        # Check if the value for the quarter is missing
        if pd.isnull(row["value(millions)"]):
            # Check if there are three other quarters in the same fiscal year
            other_quarters = df[
                (df["fiscal_year"] == row["fiscal_year"]) & (df.index != idx)
            ]["value(millions)"]
            if other_quarters.count() == 3:
                # If there are, find the corresponding annual data
                annual_value = annual_data[
                    annual_data["fiscal_year"] == row["fiscal_year"]
                ]["value(millions)"].values[0]
                # Calculate the sum of the other three quarters
                sum_of_quarters = other_quarters.sum()
                # Fill the missing value with the difference
                df.at[idx, "value(millions)"] = annual_value - sum_of_quarters
    return df


# Apply the function to the DataFrame
quarterly_data = fill_missing_quarters(quarterly_data, annual_data)
quarterly_data = quarterly_data.drop(columns=["frame"])
# Convert the 'fiscal_year' column to an integer
quarterly_data["fiscal_year"] = quarterly_data["fiscal_year"].str.replace("CY", "")
quarterly_data["fiscal_year"] = quarterly_data["fiscal_year"].astype(int)
annual_data["fiscal_year"] = annual_data["fiscal_year"].str.replace("CY", "")
annual_data["fiscal_year"] = annual_data["fiscal_year"].astype(int)
annual_data.set_index("fiscal_year", inplace=True)
annual_data.drop(columns=["accn"], inplace=True)
new_annual_order = ["value(millions)", "start", "end"]
annual_data = annual_data.reindex(columns=new_annual_order)
quarterly_data = quarterly_data.set_index(["fiscal_year", "quarter"])
new_merged_order = ["value(millions)", "start", "end"]
quarterly_data = quarterly_data.reindex(columns=new_merged_order)

# Access data for fiscal year 2022, quarter 1
data_point = quarterly_data.loc[(2022, 1)]

In [28]:
annual_data

,value(millions),start,end
fiscal_year,,,
2007,3495.0,2006-10-01,2007-09-29
2008,6119.0,2007-09-30,2008-09-27
2009,8235.0,2008-09-28,2009-09-26
2010,14013.0,2009-09-27,2010-09-25
2011,25922.0,2010-09-26,2011-09-24
2012,41733.0,2011-09-25,2012-09-29
2013,37037.0,2012-09-30,2013-09-28
2014,39510.0,2013-09-29,2014-09-27
2015,53394.0,2014-09-28,2015-09-26


In [29]:
quarterly_data

value(millions)      start        end
fiscal_year quarter                                       
2008        3                 1072.0 2008-03-30 2008-06-28
            4                    NaN 2008-06-29 2008-09-27
2009        1                 2255.0 2008-09-28 2008-12-27
            2                 1620.0 2008-12-28 2009-03-28
            3                 1828.0 2009-03-29 2009-06-27
            4                 2532.0 2009-06-28 2009-09-26
2010        1                 3378.0 2009-09-27 2009-12-26
            2                 3074.0 2009-12-27 2010-03-27
            3                 3253.0 2010-03-28 2010-06-26
            4                 4308.0 2010-06-27 2010-09-25
2011        1                 6004.0 2010-09-26 2010-12-25
            2                 5987.0 2010-12-26 2011-03-26
            3                 7308.0 2011-03-27 2011-06-25
            4                 6623.0 2011-06-26 2011-09-24
2012        1                13064.0 2011-09-25 2011-12-31
            2                11622.0 2012-01-01 2012-03-31
            3                 8824.0 2012-04-01 2012-06-30
            4                 8223.0 2012-07-01 2012-09-29
2013        1                13078.0 2012-09-30 2012-12-29
            2                 9547.0 2012-12-30 2013-03-30
            3                 6900.0 2013-03-31 2013-06-29
            4                 7512.0 2013-06-30 2013-09-28
2014        1                13072.0 2013-09-29 2013-12-28
            2                10223.0 2013-12-29 2014-03-29
            3                 7748.0 2014-03-30 2014-06-28
            4                 8467.0 2014-06-29 2014-09-27
2015        1                18024.0 2014-09-28 2014-12-27
            2                13569.0 2014-12-28 2015-03-28
            3                10677.0 2015-03-29 2015-06-27
            4                11124.0 2015-06-28 2015-09-26
2016        1                18361.0 2015-09-27 2015-12-26
            2                10516.0 2015-12-27 2016-03-26
            3                 7796.0 2016-03-27 2016-06-25
            4                 9014.0 2016-06-26 2016-09-24
2017        1                17891.0 2016-09-25 2016-12-31
            2                11029.0 2017-01-01 2017-04-01
            3                 8717.0 2017-04-02 2017-07-01
            4                10714.0 2017-07-02 2017-09-30
2018        1                20065.0 2017-10-01 2017-12-30
            2                13822.0 2017-12-31 2018-03-31
            3                11519.0 2018-04-01 2018-06-30
            4                14125.0 2018-07-01 2018-09-29
2019        1                19965.0 2018-09-30 2018-12-29
            2                11561.0 2018-12-30 2019-03-30
            3                10044.0 2019-03-31 2019-06-29
            4                13686.0 2019-06-30 2019-09-28
2020        1                22236.0 2019-09-29 2019-12-28
            2                11249.0 2019-12-29 2020-03-28
            3                11253.0 2020-03-29 2020-06-27
            4                12673.0 2020-06-28 2020-09-26
2021        1                28755.0 2020-09-27 2020-12-26
            2                23630.0 2020-12-27 2021-03-27
            3                21744.0 2021-03-28 2021-06-26
            4                20551.0 2021-06-27 2021-09-25
2022        1                34630.0 2021-09-26 2021-12-25
            2                25010.0 2021-12-26 2022-03-26
            3                19442.0 2022-03-27 2022-06-25
            4                20721.0 2022-06-26 2022-09-24
2023        1                29998.0 2022-09-25 2022-12-31
            2                24160.0 2023-01-01 2023-04-01